In [1]:
from scipy import io
import numpy as np
import os

In [2]:
patient = ['dzq','kly','lbg','llw','my','qeq','sll','swl','wfz','xjc','yc','ynb']
task = ['double01','double02','double03','simple01','simple02']

In [ ]:
for name in patient:
    print(name)

# cheatmap
 
os.mkdir(path) ## 创建file

os.listdir() ## 返回该路径的一个list


# ADJ load

In [10]:
file_name = os.listdir('E:\\DATABASE\\renzhi_BN\\EEG3\\cohmatrix')
# type(file_name)
# file_name

In [11]:
test_path = 'E:\\DATABASE\\renzhi_BN\\EEG3\\cohmatrix\\dzq_double01_coherence\\adj.mat'
data = io.loadmat(test_path)

In [18]:
# type(data)
# data 是一个描述mat文件的字典，直接访问当时在MATLAB中保存变量的名字即可
adj = data['adj'] # 数值保存以 np.ndarray 形式
# type(adj), adj.shape 
    # # adj.shape = [trail_num,1] trail
# adj[0,0].shape 
    # # adj[0,0].shape = [1,1] 我也不知道为什么，好像这里应该是任务 # 没有问题
# ===============================================================================
# 没有问题
# adj[0,0][0,0].shape 
    # # adj[0,0][0,0].shape = [33,1] 33个频段
# adj[0,0][0,0][0,0].shape 
    # # adj[0,0][0,0][0,0].shape = [18,18] 实际连接矩阵

(18, 18)

In [25]:

data_path = 'E:\\DATABASE\\renzhi_BN\\EEG3\\cohmatrix'
for p in patient:
    adj_p= []
    for t in task:
        pt_path = os.path.join(data_path, '{0}_{1}_coherence\\adj.mat'.format(p,t))
        pt_data = io.loadmat(pt_path)
        adj = pt_data['adj']
        adj_p.append(adj)
    adj_p = np.array(adj_p, dtype=object)
    np.save('E:\\DATABASE\\FirstGNN\\adjoridata\\{0}'.format(p), adj_p)

In [9]:
e_data = np.load('E:\\DATABASE\\FirstGNN\\adjoridata\\kly.npy', allow_pickle=True)
e_data.shape

(5,)

有的数据除了问题

In [4]:
import torch

In [ ]:
for j in range(5):
    e1 = e_data[j]
    count = 125
    k, k1 = 0, 0
    while k < count:  # 摘取125个样例
        if k1 < e1.shape[0]:  # trail
            for f in range(16):
                ed = e1[k1, 0][0, 0][f, 0]
                if ed.shape != (18,18):
                    ed = e1[k1-1, 0][0, 0][f, 0]
                # ed = torch.tensor(e1[k1, 0][0, 0][f, 0])
                # e_d1 = ed.reshape(-1)
                # values, indices = e_d1.topk(int(e_d1.shape[0]/4+ed.shape[0]), dim=0, largest=True, sorted=True)
            k1 += 1
            k += 1
        else:
            k1 = k1-e1.shape[0]                       
        print('class：{0}，trail: {1}, shape: {2}'.format(j,k1,ed.shape))

In [3]:
e_data[0].shape

(101, 1)

# signal

In [3]:
signal_data = io.loadmat('E:\\DATABASE\\renzhi_BN\\EEG3\\task_eeg\\task_eeg.mat')

In [4]:
# type(signal_data)
# signal_data.keys() # dict_keys(['__header__', '__version__', '__globals__', 'task_seq'])
signal_data = signal_data['task_seq']

In [8]:
# type(signal_data) # ndarray
# signal_data.shape # =[12,1]
# signal_data[0,0].shape # = [5,1]
# signal_data[0,0][0,0].shape # trail = [-1,1]
signal_data[0,0][0,0][0,0].shape # [18,-1]

(18, 244)

# 结果显示有的人的数据很有问题

    # print('p: {0} task: {1} trail: {2}      shape: {3}'.format(patient[p], t, trail, signal_data[p,0][t,0][trail,0].shape))
            # s1 = 1
        # if s1 == 1 and s2 == 0:
        #     print('p: {0} task: {1} trail_num: {2}'.format(patient[p], t, signal_data[p,0][t,0].shape[0]))
        #     s2 = 1

In [6]:
for p in range(12):
    for t in range(5):
        # print('p: {0}; task: {1}; trail_num: {2}'.format(patient[p], t, signal_data[p,0][t,0].shape[0]))
        s1,s2 = 0, 0
        for trail in range(signal_data[p,0][t,0].shape[0]):
            if signal_data[p,0][t,0][trail,0].shape[1] < 100:
                signal_data[p,0][t,0][trail,0] = signal_data[p,0][t,0][trail-1,0]

In [7]:
np.save('E:\\DATABASE\\FirstGNN\\signal\\signal.npy', signal_data)

In [3]:
# 重启内核后
signal = np.load('E:\\DATABASE\\FirstGNN\\signal\\signal.npy', allow_pickle=True)

In [5]:
# type(signal)
signal.shape

(12, 1)

# 为了适应signal，必须修改adj


for p in range(12):
    adj = np.load('E:\\DATABASE\\FirstGNN\\adjoridata\\{0}.npy'.format(patient[p]))
    for t in range(5):
        # print('p: {0}; task: {1}; trail_num: {2}'.format(patient[p], t, signal_data[p,0][t,0].shape[0]))
        for trail in range(signal_data[p,0][t,0].shape[0]):
            if signal_data[p,0][t,0][trail,0].shape[1] < 100:
                adj[t,0][trail,0] = adj[t,0][trail-1,0]